# Activity 10: Advanced web scraping and data gathering
## Build your own movie database by reading from an API
### This notebook does the following
* Retrieves and prints basic data about a movie (title entered by user) from the web (OMDB database)
* If a poster of the movie could be found, it downloads the file and saves at a user-specified location

In [27]:
import urllib.request, urllib.parse, urllib.error #urllib 3 module that can place HTTP requests and receive data from the cloud
import json # Python program to convert text file to JSON

### Load the secret API key (you have to get one from OMDB website and use that, 1000 daily limit) from a JSON file, stored in the same folder into a variable
Hint: Use **`json.loads()`**

#### Note: The following cell will not be executed in the solution notebook because the author cannot give out his private API key. 
#### Students/users/instructor will need to obtain a key and store in a JSON file. 
#### For the code's sake, we are calling this file `APIkeys.json`. But you need to store your own key in this file.

In [116]:
# Convert APIkeys.txt file into a JSON file (no JSON file available in Github repo) with API key included

## Create variable that stores APIkeys.txt file
filename = 'APIkeys.txt'
  
## Create dictionary where the API key will be stored
dict1 = {"OMDBapi":"a41b8caf"}
  
## Modify the dictionary
with open(filename) as fh:
  
    for line in fh:
  
        # reads each line and trims of extra the spaces 
        # and gives only the valid words
        command, description = line.strip().split(None, 1)
  
        dict1[command] = description.strip()


## Create JSON file from APIkeys.txt file named as APIkeys_2
json_file = open("APIkeys_2.json", "w")
json.dump(dict1, json_file, indent = 4, sort_keys = False)
json_file.close()

json_file

<_io.TextIOWrapper name='APIkeys_2.json' mode='w' encoding='cp1252'>

In [119]:
# Load the secret API key from a JSON file and store in the same folder into a variable
with open('APIkeys.json') as f:
    keys = json.load(f)
    omdbapi = keys['OMDBapi']

omdbapi

'a41b8caf'

### The final URL to be passed should look like: http://www.omdbapi.com/?t=movie_name&apikey=secretapikey 
Do the following,
* Assign the OMDB portal (http://www.omdbapi.com/?) as a string to a variable `serviceurl` (don't miss the `?`)
* Create a variable `apikey` with the last portion of the URL ("&apikey=secretapikey"), where `secretapikey` is your own API key (an actual code)
* The movie name portion i.e. "t=movie_name" will be addressed later

In [133]:
serviceurl = 'http://www.omdbapi.com/?' # assign OMDB portal as string to variable serviceurl
apikey = '&apikey='+omdbapi # created variable apikey with last portion of URL is the OMBDapi api key

apikey

'&apikey=a41b8caf'

### Write a utility function `print_json` to print nicely the movie data from a JSON file (which we will get from the portal)
Here are the keys of a JSON file,

'Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 'Actors', 'Plot', 'Language','Country', 'Awards', 'Ratings', 'Metascore', 'imdbRating', 'imdbVotes', 'imdbID'

In [134]:
def print_json(json_data): # created print_json function to nicely print out the data from a JSON file
    list_keys=['Title', 'Year', 'Rated', 'Released', 'Runtime', 'Genre', 'Director', 'Writer', 
               'Actors', 'Plot', 'Language', 'Country', 'Awards', 'Ratings', 
               'Metascore', 'imdbRating', 'imdbVotes', 'imdbID'] # created list of the keys listed above in the instructions
    print("-"*50) # print 50
    for k in list_keys: # for loop of list_keys list
        if k in list(json_data.keys()):
            print(f"{k}: {json_data[k]}")
    print("-"*50)

### Write a utility function to download a poster of the movie based on the information from the jason dataset and save in your local folder

* Use `os` module
* The poster data is stored in the JSON key 'Poster'
* You may want to split the name of the Poster file and extract the file extension only. Let's say the extension is ***'jpg'***.
* Then later join this extension to the movie name and create a filename like ***movie.jpg***
* Use the Python command `open` to open a file and write the poster data. Close the file after done.
* This function may not return anything. It just saves the poster data as an image file.

In [135]:
# save_poster function to download a poster of the movie based on the info from the json dataset and save to local folder
def save_poster(json_data):
    import os # import os module
    title = json_data['Title']
    poster_url = json_data['Poster']
    # Splits the poster url by '.' and picks up the last string as file extension
    poster_file_extension=poster_url.split('.')[-1]
    # Reads the image file from web
    poster_data = urllib.request.urlopen(poster_url).read()
        
    savelocation=os.getcwd()+'\\'+'Posters'+'\\'
    # Creates new directory if the directory does not exist. Otherwise, just use the existing path.
    if not os.path.isdir(savelocation):
        os.mkdir(savelocation)
    
    filename=savelocation+str(title)+'.'+poster_file_extension
    f=open(filename,'wb')
    f.write(poster_data)
    f.close()

### Write a utility function `search_movie` to search a movie by its name, print the downloaded JSON data (use the `print_json` function for this) and save the movie poster in the local folder (use `save_poster` function for this)

* Use `try-except` loop for this i.e. try to connect to the web portal, if successful proceed but if not (i.e. exception raised) then just print an error message
* Here use the previously created variables `serviceurl` and `apikey`
* You have to pass on a dictionary with a key `t` and the movie name as the corresponding value to `urllib.parse.urlencode()` function and then add the `serviceurl` and `apikey` to the output of the function to construct the full URL
* This URL will be used for accessing the data
* The JSON data has a key called `Response`. If it is `True`, that means the read was successful. Check this before processing the data. If not successful, then print the JSON key `Error`, which will contain the appropriate error message returned by the movie database.

In [136]:
# search_movie function to search a movie by its name, and print the downloaded JSON data using print_json function. 
def search_movie(title):
    try:
        url = serviceurl + urllib.parse.urlencode({'t': str(title)})+apikey # use apikey that contains my API key
        print(f'Retrieving the data of "{title}" now... ')
        print(url)
        uh = urllib.request.urlopen(url)
        data = uh.read()
        json_data=json.loads(data)
        
        if json_data['Response']=='True':
            print_json(json_data)
            # Asks user whether to download the poster of the movie
            if json_data['Poster']!='N/A':
                save_poster(json_data)
        else:
            print("Error encountered: ",json_data['Error'])
    
    except urllib.error.URLError as e:
        print(f"ERROR: {e.reason}")

### Test `search_movie` function by entering *Titanic*

In [137]:
# Call the search_movie function and pass the movie title "Titanic" which also downloads the poster
search_movie("Titanic")

Retrieving the data of "Titanic" now... 
http://www.omdbapi.com/?t=Titanic&apikey=a41b8caf
--------------------------------------------------
Title: Titanic
Year: 1997
Rated: PG-13
Released: 19 Dec 1997
Runtime: 194 min
Genre: Drama, Romance
Director: James Cameron
Writer: James Cameron
Actors: Leonardo DiCaprio, Kate Winslet, Billy Zane
Plot: A seventeen-year-old aristocrat falls in love with a kind but poor artist aboard the luxurious, ill-fated R.M.S. Titanic.
Language: English, Swedish, Italian, French
Country: United States, Mexico, Australia
Awards: Won 11 Oscars. 125 wins & 83 nominations total
Ratings: [{'Source': 'Internet Movie Database', 'Value': '7.8/10'}, {'Source': 'Rotten Tomatoes', 'Value': '89%'}, {'Source': 'Metacritic', 'Value': '75/100'}]
Metascore: 75
imdbRating: 7.8
imdbVotes: 1,098,236
imdbID: tt0120338
--------------------------------------------------


### Test `search_movie` function by entering "*Random_error*" (obviously this will not be found and you should be able to check whether your error catching code is working properly)

In [138]:
# Call the search_movie function and pass the the string "Random_error" to check that it won't pull random names
search_movie("Random_error")

Retrieving the data of "Random_error" now... 
http://www.omdbapi.com/?t=Random_error&apikey=a41b8caf
Error encountered:  Movie not found!


### Look for a folder called 'Posters' in the same directory you are working in. It should contain a file called 'Titanic.jpg'. Open and see if the poster came alright!

In [173]:
# The Titanic movie poster did download to my Jupyter Notebook space!

# Imports PIL module 
from PIL import Image
  
# open method used to open different extension image file
im = Image.open(r"C:\Users\kadams\Downloads\DSC540_Packt-Data_Wrangling-master\Packt-Data_Wrangling-master\Weekly Exercises\Posters\Titanic.jpg") 
  
# This method will show image in any image viewer
im.show() 

## The image was displayed in the Photos program on my local computer!